  0%|          | 0/1000 [00:00<?, ?it/s]


NameError: ignored

In [3]:
# !pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 4.4 MB/s eta 0:00:00


In [4]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

import gymnasium as gym

In [9]:
# Agent
from keras.models import Model, Sequential
from tensorflow.keras.activations import linear

from keras.layers import Dense, Input
from keras.optimizers import Adam
from collections import deque
import gc

print("Change back to 1.0 epsilon")
class DDQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size

        self.tau = 0.5
        self.batch_size = 100
        self.memory = deque(maxlen=10000)
        self.gamma = 0.99
        self.epsilon = 0.1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.double_dqn = True

    def build_model(self):
        state_size = 4  # Replace with your actual state size
        action_size = 2  # Replace with the number of actions

        model = Sequential()
        model.add(Dense(16, input_shape=(state_size,), activation="linear"))
        model.add(Dense(action_size, activation="linear"))
        model.compile(loss="mse", optimizer="adam")

        # # model.add(Dense(128, input_dim = self.state_size, activation='relu'))
        # # model.add(Dense(64, activation='relu'))
        # # model.add(Dense(32, activation='relu'))
        # model.add(Dense(self.action_size, activation='relu'))
        # model.compile(loss='mse',optimizer=Adam(learning_rate=self.learning_rate))
        # super(Model, self).__init__()
        return model

    def remember(self, state, action, reward, next_state, done):
      # Reshape state and next state before adding them to memory
      state = np.reshape(state, [1, self.state_size])
      next_state = np.reshape(next_state, [1, self.state_size])
      self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # if self.epsilon > self.epsilon_min:
        #     self.epsilon *= self.epsilon_decay
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def replay(self):
        minibatch = self.sample(self.batch_size)
        for state, action, reward, next_state, done in minibatch:
            self.update_target_model()
            target = self.model.predict(state, verbose=0)
            if not done:
                # if self.double_dqn:
                    predicted_action = np.argmax(target[0])#self.model.predict(next_state,verbose=0)[0])
                    target_q = self.target_model.predict(next_state, verbose=0)[0][predicted_action]
                    target[0][action] = reward + self.gamma * target_q
                # else:
                #     target_q = self.target_model.predict(next_state, verbose=0)[0]
                #     target[0][action] = reward + self.gamma * max(target_q)
            else:
                target[0][action] = reward
            self.model.fit(state,target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        gc.collect()

    def update_target_model(self):
        #TODO: Try training without a target network
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1- self.tau)
        self.target_model.set_weights(target_weights)

    def save(self, file):
        self.model.save_weights(file)

    def load(self, file):
        self.model.load_weights(file)
        self.target_model.load_weights(file)
    def update_target_model(self, tau=0.01):
      """
      Updates the target network weights using polyak averaging.

      Args:
        tau: Mixing factor for polyak averaging (generally between 0.001 and 0.1).
      """
      main_weights = self.model.get_weights()
      target_weights = self.target_model.get_weights()
      for i in range(len(main_weights)):
        target_weights[i] = tau * main_weights[i] + (1 - tau) * target_weights[i]
      self.target_model.set_weights(target_weights)



Change back to 1.0 epsilon


In [ ]:
# Train
env = gym.make("CartPole-v1")
state_size_, action_size_ = 4, 2
dqn_agent = DDQNAgent(state_size_, action_size_)
try:
    dqn_agent.load('cartpole.h5')
except:
    print("No previous model detected")
n_episodes = 1000
n_steps = 200
# capacity = 10000

streak_step = 0
total_step = 0

save_episodes = 10
for episode in tqdm(range(n_episodes)):
    cur_state_, _ = env.reset()
    for step in range(n_steps):
        streak_step += 1
        total_step += 1
        cur_state_ = np.reshape(cur_state_, [1, state_size_])
        action = dqn_agent.act(cur_state_)
        observation, reward, done, _, _ = env.step(action)
        observation = np.reshape(observation, [1, state_size_])
        if done:
            reward = -5
        dqn_agent.remember(cur_state_, action, reward, observation, done)
        if total_step % dqn_agent.batch_size == 0:
            dqn_agent.replay()
            dqn_agent.update_target_model()
        cur_state_ = observation
        if done:
            break
    if (episode + 1) % save_episodes == 0:
        dqn_agent.save("cartpole.h5")
        print(f"Avg game: {streak_step / save_episodes}, Epsilon: {dqn_agent.epsilon}")
        streak_step = 0

  1%|          | 10/1000 [00:08<13:17,  1.24it/s]

Avg game: 9.9, Epsilon: 0.1


  2%|▏         | 20/1000 [00:41<16:53,  1.03s/it]

Avg game: 9.4, Epsilon: 0.099


  3%|▎         | 30/1000 [01:15<16:52,  1.04s/it]

...

 69%|██████▉   | 690/1000 [51:08<33:22,  6.46s/it]

Avg game: 27.8, Epsilon: 0.04171208799332206


 70%|███████   | 700/1000 [51:47<15:49,  3.17s/it]

Avg game: 9.8, Epsilon: 0.041294967113388835


 70%|███████   | 705/1000 [51:52<06:59,  1.42s/it]

In [8]:
# # Doesn't render in jupyter
# obs = env.reset()
# done = False
# while not done:
#     action = dqn_agent.act(obs)
#     obs, rewards, done, _, _ = env.step(action)
#     env.render()
# env.close()

ValueError: ignored

In [6]:
from torch import nn
from torch.nn import functional


class Model(nn.Module):
    def __init__(self, input_features, output_values):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features=input_features, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=32)
        self.fc3 = nn.Linear(in_features=32, out_features=output_values)

    def forward(self, x):
        x = functional.selu(self.fc1(x))
        x = functional.selu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import gymnasium as gym
from collections import deque
import random

# Parameters
use_cuda = True
episode_limit = 100
target_update_delay = 2  # update target net every target_update_delay episodes
test_delay = 10
learning_rate = 1e-4
epsilon = 1  # initial epsilon
min_epsilon = 0.1
epsilon_decay = 0.9 / 2.5e3
gamma = 0.99
memory_len = 10000

env = gym.make('CartPole-v1')
n_features = len(env.observation_space.high)
n_actions = env.action_space.n

memory = deque(maxlen=memory_len)
# each memory entry is in form: (state, action, env_reward, next_state)
device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()
policy_net = Model(n_features, n_actions).to(device)
target_net = Model(n_features, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()


def get_states_tensor(sample, states_idx):
    sample_len = len(sample)
    states_tensor = torch.empty((sample_len, n_features), dtype=torch.float32, requires_grad=False)

    features_range = range(n_features)
    for i in range(sample_len):
        for j in features_range:
            states_tensor[i, j] = sample[i][states_idx][j].item()

    return states_tensor


def normalize_state(state):
    # pass
    state[0] /= 2.5
    state[1] /= 2.5
    state[2] /= 0.3
    state[3] /= 0.3


def state_reward(state, env_reward):
    # return env_reward
    return env_reward - (abs(state[0]) + abs(state[2])) / 2.5


def get_action(state, e=min_epsilon):
    if random.random() < e:
        # explore
        action = random.randrange(0, n_actions)
    else:
        state = torch.tensor(state, dtype=torch.float32, device=device)
        action = policy_net(state).argmax().item()

    return action


def fit(model, inputs, labels):
    inputs = inputs.to(device)
    labels = labels.to(device)
    train_ds = TensorDataset(inputs, labels)
    train_dl = DataLoader(train_ds, batch_size=5)

    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.train()
    total_loss = 0.0

    for x, y in train_dl:
        out = model(x)
        loss = criterion(out, y)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    return total_loss / len(inputs)


def optimize_model(train_batch_size=100):
    train_batch_size = min(train_batch_size, len(memory))
    train_sample = random.sample(memory, train_batch_size)

    state = get_states_tensor(train_sample, 0)
    next_state = get_states_tensor(train_sample, 3)

    q_estimates = policy_net(state.to(device)).detach()
    next_state_q_estimates = target_net(next_state.to(device)).detach()
    next_actions = policy_net(next_state.to(device)).argmax(dim=1)

    for i in range(len(train_sample)):
        next_action = next_actions[i].item()
        q_estimates[i][train_sample[i][1]] = (state_reward(next_state[i], train_sample[i][2]) +
                                              gamma * next_state_q_estimates[i][next_action].item())

    fit(policy_net, state, q_estimates)


def train_one_episode():
    global epsilon
    current_state, _ = env.reset()
    normalize_state(current_state)
    done = False
    score = 0
    reward = 0
    while not done:
        action = get_action(current_state, epsilon)
        next_state, env_reward, done, _, _ = env.step(action)
        normalize_state(next_state)
        memory.append((current_state, action, env_reward, next_state))
        current_state = next_state
        score += env_reward
        reward += state_reward(next_state, env_reward)

        optimize_model(100)

        epsilon -= epsilon_decay

    return score, reward


def test():
    state, _ = env.reset()
    normalize_state(state)
    done = False
    score = 0
    reward = 0
    while not done:
        action = get_action(state)
        state, env_reward, done, _, _ = env.step(action)
        normalize_state(state)
        score += env_reward
        reward += state_reward(state, env_reward)

    return score, reward


def main():
    best_test_reward = 0

    for i in range(episode_limit):
        score, reward = train_one_episode()

        print(f'Episode {i + 1}: score: {score} - reward: {reward}')

        if i % target_update_delay == 0:
            target_net.load_state_dict(policy_net.state_dict())
            target_net.eval()

        if (i + 1) % test_delay == 0:
            test_score, test_reward = test()
            print(f'Test Episode {i + 1}: test score: {test_score} - test reward: {test_reward}')
            if test_reward > best_test_reward:
                print('New best test reward. Saving model')
                best_test_reward = test_reward
                torch.save(policy_net.state_dict(), 'policy_net.pth')

    if episode_limit % test_delay != 0:
        test_score, test_reward = test()
        print(f'Test Episode {episode_limit}: test score: {test_score} - test reward: {test_reward}')
        if test_reward > best_test_reward:
            print('New best test reward. Saving model')
            best_test_reward = test_reward
            torch.save(policy_net.state_dict(), 'policy_net.pth')

    print(f'best test reward: {best_test_reward}')


if __name__ == '__main__':
    main()

Episode 1: score: 24.0 - reward: 21.92471436560154
Episode 2: score: 26.0 - reward: 21.789406988024712
Episode 3: score: 12.0 - reward: 10.321745324134827
Episode 4: score: 37.0 - reward: 33.57567148283125
Episode 5: score: 24.0 - reward: 21.02101508677006
Episode 6: score: 25.0 - reward: 22.066822504997248
Episode 7: score: 49.0 - reward: 45.11534642204642
Episode 8: score: 19.0 - reward: 17.188667308539156
Episode 9: score: 20.0 - reward: 18.24884291291237
Episode 10: score: 30.0 - reward: 26.157555196434263
Test Episode 10: test score: 51.0 - test reward: 43.27161414027214
New best test reward. Saving model
Episode 11: score: 14.0 - reward: 12.515208195149897
Episode 12: score: 21.0 - reward: 19.12148921638727
Episode 13: score: 33.0 - reward: 27.46333811879159
Episode 14: score: 41.0 - reward: 35.4634566962719
Episode 15: score: 27.0 - reward: 24.561720565706494
Episode 16: score: 24.0 - reward: 19.944308507442475
Episode 17: score: 12.0 - reward: 10.114508500695226
Episode 18: sco

KeyboardInterrupt: ignored